# Master DSLS / Programming 3 / Assignment 5
# Mapreduce & PySpark Assignment

## 1. Goal

The goal of this assignment is to read in a large dataset of protein annotation information and to manipulate, summarize and analyze it using PySpark Dataframes.

Protein annotation is a branch of bioinformatics which classifies the different parts of a protein's structure based on both sequence and functional characteristics. For instance, it recognizes structural elements like trans-membrane helices, but also particular active sites ("Serine Protease") and also signal peptides ("periplasmic membrane tag"). The holy grail of this field is to use these different annotations of parts of the protein sequence, and to combine them to predict the function of the protein as a whole. (Without having to carry out actual experiments in the lab !)

The subject is the output of the InterProScan protein annotation service [InterproScan online](http://www.ebi.ac.uk/interpro/), [NAR article](https://academic.oup.com/nar/article/49/D1/D344/5958491) Briefly, InterPROscan is a meta-annotator; it runs different protein function annotators in turn on an input amino-acid sequence FASTA file and collects the output of each, labelling them with a unique and consistent identifier; the "InterPRO number". I used this service to annotate all currently known prokaryotic (Bacteria, Archaea) genomes to investigate better methods of metagenomics sequence annotation.

# 2. Deliverables

You need to write a script called `assignment5.py` in your `Assignment5` folder in your `programming3` GitHub repository. This script takes as input an InterPROscan output file; you can test on the example data in the /data/dataprocessing/interproscan/all_bacilli.tsv file on assemblix2012 and assemblix2019. You should use the PySpark Dataframe interface to read in and manipulate this file. This file contains ~4,200,000 protein annotations. You need to use the PySpark dataframe functions to answer the following questions:
1. How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?
2. How many annotations does a protein have on average?
3. What is the most common GO Term found?
4. What is the average size of an InterPRO feature found in the dataset?
5. What is the top 10 most common InterPRO features?
6. If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?
7. If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?
8. And the top 10 least common?
9. Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?
10. What is the coefficient of correlation ($R^2$) between the size of the protein and the number of features found?

Your output should be a CSV file with 3 columns;
1. in the first column the question number
2. in the second column the answer(s) to the question
3. in the third column the output of the scheduler's physical plan (using the `.explain()` PySpark method) as a string

NB1: Make sure you use the /commons/conda environment
NB2: Use only 16 threads maximum; `sc = SparkContext('local[16]')`
NB3: Use the `csv` Python module to make the CSV file in "excel" format; this makes it easier to deal with the different answer types (number, string, list etc.)

In [1]:
# Output format : https://interproscan-docs.readthedocs.io/en/latest/OutputFormats.html
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [2]:
schema = StructType([
    StructField("Protein_accession", StringType(), True),
    StructField("Sequence_MD5_digest", StringType(), True),
    StructField("Sequence_length", IntegerType(), True),
    StructField("Analysis", StringType(), True),
    StructField("Signature_accession", StringType(), True),
    StructField("Signature_description", StringType(), True),
    StructField("Start_location", IntegerType(), True),
    StructField("Stop_location", IntegerType(), True),
    StructField("Score", FloatType(), True),
    StructField("Status", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("InterPro_annotations_accession", StringType(), True),
    StructField("InterPro_annotations_description", StringType(), True),
    StructField("GO_annotations", StringType(), True),
    StructField("Pathways_annotations", StringType(), True)])
spark = SparkSession.builder.master("local[16]").appName("InterPro").getOrCreate()
df = spark.read.option("sep","\t").option("header","False").csv("/data/dataprocessing/interproscan/all_bacilli.tsv",schema=schema)

In [3]:
# 1. How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?
data1 = df.select('InterPro_annotations_accession')\
          .filter(df.InterPro_annotations_accession != "-")\
          .distinct()\
          .count()


In [4]:
# 2. How many annotations does a protein have on average?
data2 = df.select("Protein_accession",'InterPro_annotations_accession')\
            .filter(df.InterPro_annotations_accession != "-")\
            .groupBy("Protein_accession")\
            .count()\
            .select(mean("count"))\
            .collect()[0].__getitem__(0)


In [5]:
# 3. What is the most common GO Term found?
data3 = df.select(df.GO_annotations, explode(split(col("GO_annotations"),"\|"))\
                    .alias("Split_col"))
data3 = data3.filter(data3.Split_col != "-")\
            .select("Split_col")\
            .groupby("Split_col")\
            .count()\
            .sort("count",ascending=False)\
            .collect()[:10]
data3 = [data[0] for data in data3]


In [6]:
# 4. What is the average size of an InterPRO feature found in the dataset?
data4 = df.withColumn('Sub', ( df['Stop_location'] - df['Start_location']))\
            .summary("mean")\
            .collect()[0]\
            .__getitem__(-1)


In [7]:
# 5. What is the top 10 most common InterPRO features?
data5 = df.select('InterPro_annotations_accession')\
            .filter(df.InterPro_annotations_accession != "-")\
            .groupBy('InterPro_annotations_accession')\
            .count()\
            .sort("count",ascending=False)\
            .select("InterPro_annotations_accession")\
            .collect()[:10]
data5 = [data[0] for data in data5]


In [8]:
# 6. If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?
data6 = df.select('InterPro_annotations_accession',"Sequence_length")\
            .filter((df['Stop_location'] - df['Start_location'])>=0.9)\
            .filter(df.InterPro_annotations_accession != "-")\
            .groupBy('InterPro_annotations_accession')\
            .count()\
            .sort("count",ascending=False)\
            .select("InterPro_annotations_accession")\
            .collect()[:10]
data6 = [data[0] for data in data6]


In [9]:
# 7. If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?
data7 = df.select(df.InterPro_annotations_description,explode(split(col("InterPro_annotations_description"),"\.|/| |,"))\
            .alias("Split_col"))
data7 = data7.select("Split_col")\
            .filter(data7.Split_col != "")\
            .groupby("Split_col")\
            .count()\
            .sort("count",ascending=False)\
            .select("Split_col").collect()[:10]
data7 = [data[0] for data in data7]


In [10]:
# 8. And the top 10 least common?
data8 = df.select(df.InterPro_annotations_description,explode(split(col("InterPro_annotations_description"),"\.|/| |,"))\
            .alias("Split_col"))
data8 = data8.select("Split_col")\
            .filter(data8.Split_col != "")\
            .groupby("Split_col")\
            .count()\
            .sort("count",ascending=True)\
            .select("Split_col").collect()[:10]
data8 = [data[0] for data in data8]


In [11]:
# 9. Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?
data9 = df.select('InterPro_annotations_accession',"Sequence_length")\
            .filter(df.InterPro_annotations_accession != "-")\
            .groupBy('InterPro_annotations_accession')\
            .count()\
            .sort("count",ascending=False)\
            .select("InterPro_annotations_accession")\
            .collect()[:1][0][0]
data9 = df.select(df.InterPro_annotations_accession,df.InterPro_annotations_description)\
            .filter(df.InterPro_annotations_accession == data9)\
            .collect()[0][1]
data9 = data9.split()


In [12]:
# 10. What is the coefficient of correlation ($R^2$) between the size of the protein and the number of features found?
data10=df.dropDuplicates(['InterPro_annotations_accession'])\
            .filter(df.InterPro_annotations_accession != "-")\
            .select(["InterPro_annotations_accession","Start_location","Stop_location"])\
            .withColumn('Sub', ( df['Stop_location'] - df['Start_location']))
data11=df.select("InterPro_annotations_accession")\
            .filter(df.InterPro_annotations_accession != "-")\
            .groupby("InterPro_annotations_accession",)\
            .count()
data10 = data10.join(data11,"InterPro_annotations_accession")
data10 = data10.corr('Sub', 'count')

In [14]:
df.explain()

== Physical Plan ==
FileScan csv [Protein_accession#0,Sequence_MD5_digest#1,Sequence_length#2,Analysis#3,Signature_accession#4,Signature_description#5,Start_location#6,Stop_location#7,Score#8,Status#9,Date#10,InterPro_annotations_accession#11,InterPro_annotations_description#12,GO_annotations#13,Pathways_annotations#14] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/data/dataprocessing/interproscan/all_bacilli.tsv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Protein_accession:string,Sequence_MD5_digest:string,Sequence_length:int,Analysis:string,Si...




## 3. Extra Information
- The PySpark documentation; https://spark.apache.org/docs/latest/api/python/index.html
- The InterProScan documentation; https://interpro-documentation.readthedocs.io/en/latest/interproscan.html
- Nice Spark tutorial; http://ampcamp.berkeley.edu/big-data-mini-course/data-exploration-using-spark.html
- About Protein annotation; Paper on PFAM, Paper on TIGRFAM